In [1]:
import gradio as gr
import json
import os

# Install anthropic if not already installed
try:
    import anthropic
except ImportError:
    !pip install anthropic
    import anthropic

# Initialize Anthropic client (API key should be set in environment)
# For demo purposes, we'll use a mock function
def analyze_conversation(conversation_text):
    """
    Analyzes conversation for emotional intelligence insights
    """
    if not conversation_text.strip():
        return {
            "error": "Please enter a conversation to analyze"
        }

    # In production, you would use:
    # client = anthropic.Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))

    prompt = f"""You are SoulSync AI, an emotional intelligence coach. Analyze this conversation and provide insights in JSON format.

Conversation:
{conversation_text}

Provide a JSON response with:
1. connectionScore (0-100): Overall emotional connection quality
2. emotionalTone: Brief description of emotions present
3. communicationGaps: Array of 2-3 things being missed
4. suggestions: Array of 3 specific ways to deepen connection
5. empathyInsights: Object with insights for each person
6. conversationHealth: Object with strengths and improvements arrays

Respond ONLY with valid JSON, no markdown formatting."""

    try:
        # Mock response for demo (replace with actual API call)
        # Uncomment below for real implementation:
        """
        message = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=1500,
            messages=[{"role": "user", "content": prompt}]
        )
        response_text = message.content[0].text
        """

        # Mock response based on conversation analysis
        response_text = generate_mock_analysis(conversation_text)

        # Parse JSON response
        analysis = json.loads(response_text)

        return format_analysis_output(analysis)

    except Exception as e:
        return {"error": f"Analysis failed: {str(e)}"}


def generate_mock_analysis(conversation):
    """Generate mock analysis for demo purposes"""
    # Analyze conversation sentiment
    words = conversation.lower()

    # Calculate connection score based on conversation patterns
    score = 50
    if "feel" in words or "understand" in words:
        score += 15
    if "?" in conversation:
        score += 10
    if "fine" in words and len(conversation.split('\n')) < 5:
        score -= 20
    if "upset" in words or "tired" in words:
        score -= 10

    score = max(20, min(95, score))

    # Generate contextual analysis
    analysis = {
        "connectionScore": score,
        "emotionalTone": "Guarded with underlying tension. Surface-level responses suggest emotional distance.",
        "communicationGaps": [
            "One person is asking questions but not receiving meaningful answers",
            "Emotions are being denied rather than explored",
            "There's a lack of vulnerability and openness in the exchange"
        ],
        "suggestions": [
            "Try reflecting feelings: 'It sounds like you might be carrying something heavy today'",
            "Share your own vulnerability first to create a safe space",
            "Ask open-ended questions like 'What's been on your mind lately?'"
        ],
        "empathyInsights": {
            "Person asking questions": "Sensing something is wrong and trying to connect, but may feel shut out or frustrated",
            "Person giving short answers": "Likely overwhelmed or not ready to share, possibly feeling defensive or protective"
        },
        "conversationHealth": {
            "strengths": [
                "One person is showing care by checking in",
                "There's an attempt to go deeper than surface conversation"
            ],
            "improvements": [
                "Create more emotional safety before probing",
                "Practice active listening without judgment",
                "Allow for silence and processing time"
            ]
        }
    }

    return json.dumps(analysis)


def format_analysis_output(analysis):
    """Format the analysis into readable text output"""
    if "error" in analysis:
        return analysis["error"]

    output = f"""
# 🎯 CONNECTION SCORE: {analysis['connectionScore']}/100

## 💭 Emotional Tone
{analysis['emotionalTone']}

## ⚠️ Communication Gaps
"""
    for gap in analysis['communicationGaps']:
        output += f"• {gap}\n"

    output += "\n## 💡 Suggestions to Deepen Connection\n"
    for i, suggestion in enumerate(analysis['suggestions'], 1):
        output += f"{i}. {suggestion}\n"

    output += "\n## ❤️ What They're Really Feeling\n"
    for person, insight in analysis['empathyInsights'].items():
        output += f"**{person}:** {insight}\n\n"

    output += "## 📊 Conversation Health\n\n**✅ Strengths:**\n"
    for strength in analysis['conversationHealth']['strengths']:
        output += f"• {strength}\n"

    output += "\n**🎯 Areas to Improve:**\n"
    for improvement in analysis['conversationHealth']['improvements']:
        output += f"• {improvement}\n"

    return output


# Sample conversations
SAMPLE_1 = """Person A: "How was your day?"
Person B: "Fine."
Person A: "Just fine? You seem upset."
Person B: "I'm not upset. I'm just tired."
Person A: "Are you sure? You can talk to me."
Person B: "There's nothing to talk about." """

SAMPLE_2 = """Person A: "Hey! It's been so long! How have you been?"
Person B: "Oh, you know, busy with work and everything."
Person A: "I've missed hanging out! We should catch up properly."
Person B: "Yeah, that would be nice. Things have been... different lately."
Person A: "Different how? Is everything okay?" """

SAMPLE_3 = """Manager: "I noticed you've been quiet in meetings lately."
Employee: "Yeah, I've just been focused on my tasks."
Manager: "Is there something on your mind? Your contributions are valuable."
Employee: "I appreciate that. I guess I've been feeling a bit overwhelmed."
Manager: "Let's talk about how we can help with that." """


# Create Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), css="""
    .gradio-container {
        max-width: 1200px !important;
    }
    .header {
        text-align: center;
        padding: 20px;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border-radius: 10px;
        margin-bottom: 20px;
    }
""") as demo:

    gr.HTML("""
        <div class="header">
            <h1>💜 SoulSync AI - Emotional Intelligence Assistant</h1>
            <p>AI that makes human conversations deeper, not shallower</p>
        </div>
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## 📝 Enter Conversation")
            conversation_input = gr.Textbox(
                label="Conversation Text",
                placeholder="Paste a conversation here...\n\nExample:\nPerson A: 'How are you?'\nPerson B: 'Fine.'",
                lines=15,
                value=""
            )

            analyze_btn = gr.Button("🧠 Analyze Emotional Intelligence", variant="primary", size="lg")

            gr.Markdown("### 🎯 Try a Sample:")
            with gr.Row():
                sample1_btn = gr.Button("Couple's Discussion", size="sm")
                sample2_btn = gr.Button("Friends Reconnecting", size="sm")
                sample3_btn = gr.Button("Manager & Employee", size="sm")

        with gr.Column(scale=1):
            gr.Markdown("## 💡 AI Insights & Coaching")
            analysis_output = gr.Markdown(
                value="*Your emotional intelligence insights will appear here...*",
                label="Analysis Results"
            )

    gr.Markdown("""
    ---
    ### 🌟 How SoulSync Works

    **Real-Time Analysis** • Understands emotional undertones and communication patterns
    **Empathy Coaching** • Helps you recognize what others are truly feeling
    **Connection Growth** • Actionable insights to deepen your relationships

    *Built with Claude Sonnet 4 - AI that makes us more human, not less.*
    """)

    # Event handlers
    analyze_btn.click(
        fn=analyze_conversation,
        inputs=[conversation_input],
        outputs=[analysis_output]
    )

    sample1_btn.click(
        fn=lambda: SAMPLE_1,
        outputs=[conversation_input]
    )

    sample2_btn.click(
        fn=lambda: SAMPLE_2,
        outputs=[conversation_input]
    )

    sample3_btn.click(
        fn=lambda: SAMPLE_3,
        outputs=[conversation_input]
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.5/371.5 kB 9.8 MB/s eta 0:00:00
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://050ca9f47fd15dc8b9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
